In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## 1.모델 가져오기(git clone)

In [2]:
%pwd

'/content'

In [3]:
%cd /content/drive/MyDrive/yolo/pklot detection

/content/drive/MyDrive/yolo/pklot detection


In [4]:
%pwd

'/content/drive/MyDrive/yolo/pklot detection'

In [6]:
!git clone https://github.com/ultralytics/yolov5

Cloning into 'yolov5'...
remote: Enumerating objects: 16582, done.
remote: Counting objects: 100% (60/60), done.
remote: Compressing objects: 100% (41/41), done.
remote: Total 16582 (delta 30), reused 39 (delta 19), pack-reused 16522
Receiving objects: 100% (16582/16582), 15.11 MiB | 12.93 MiB/s, done.
Resolving deltas: 100% (11383/11383), done.


In [5]:
%cd /content/drive/MyDrive/yolo/pothole detection/yolov5

/content/drive/MyDrive/yolo/pothole detection/yolov5


In [ ]:
!pip install -r requirements.txt
import torch
from IPython.display import Image, clear_output  # to display images
# clear_output()
print('Setup complete. Using torch %s %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'))

## 2.데이터셋 다운로드

In [7]:
%cd /content/drive/MyDrive/yolo/pklot detection/yolov5/pklot

/content/drive/MyDrive/yolo/pklot detection/yolov5/pklot


In [18]:
%pwd

'/content/drive/MyDrive/yolo/pklot detection/yolov5/pklot'

In [ ]:
!curl -L "https://public.roboflow.com/ds/Inm0XNYyuF?key=FUK19IOAjZ" > roboflow.zip; unzip roboflow.zip; rm roboflow.zip

### dataSet의 구성 내용을 확인

In [12]:
from glob import glob
train_img_list = glob("/content/drive/MyDrive/yolo/pklot detection/yolov5/pklot/train/images/*")
val_img_list = glob("/content/drive/MyDrive/yolo/pklot detection/yolov5/pklot/valid/images/*")
test_img_list = glob("/content/drive/MyDrive/yolo/pklot detection/yolov5/pklot/test/images/*")
print(len(train_img_list), len(val_img_list), len(test_img_list))

8691 2483 1242


In [ ]:
import yaml
with open("/content/drive/MyDrive/yolo/pklot detection/yolov5/pklot/train.txt",'w') as f:
  f.write('\n'.join(train_img_list) + '\n')
with open("/content/drive/MyDrive/yolo/pklot detection/yolov5/pklot/val.txt",'w') as f:
  f.write('\n'.join(val_img_list) + '\n')
with open("/content/drive/MyDrive/yolo/pklot detection/yolov5/pklot/test.txt",'w') as f:
  f.write('\n'.join(test_img_list) + '\n')

##3. yaml파일 만들기

In [14]:
from IPython.core.magic import register_line_cell_magic

@register_line_cell_magic
def writetemplate(line,cell):
    with open(line, 'w') as f:
        f.write(cell.format(**globals()))

In [ ]:
%cat data.yaml

In [20]:
%%writetemplate /content/drive/MyDrive/yolo/pklot detection/yolov5/pklot/data.yaml

train: ./pklot/train/images
val: ./pklot/valid/images

nc: 2
names: ['space-empty', 'space-occupied']

In [ ]:
%cat data.yaml

##4.FineTunning

In [24]:
with open('data.yaml','r') as stream:
  num_classes = str(yaml.safe_load(stream)['nc'])
  print (num_classes)

2


In [26]:
%pwd

'/content/drive/MyDrive/yolo/pklot detection/yolov5/pklot'

In [30]:
%cd /content/drive/MyDrive/yolo/pklot detection/yolov5/models

/content/drive/MyDrive/yolo/pklot detection/yolov5/models


In [32]:
%pwd

'/content/drive/MyDrive/yolo/pklot detection/yolov5/models'

In [ ]:
%cat yolov5s.yaml

In [36]:
%%writetemplate custom_yolov5s.yaml

# Parameters
nc: {num_classes} # number of classes
depth_multiple: 0.33 # model depth multiple
width_multiple: 0.50 # layer channel multiple
anchors:
  - [10, 13, 16, 30, 33, 23] # P3/8
  - [30, 61, 62, 45, 59, 119] # P4/16
  - [116, 90, 156, 198, 373, 326] # P5/32

# YOLOv5 v6.0 backbone
backbone:
  # [from, number, module, args]
  [
    [-1, 1, Conv, [64, 6, 2, 2]], # 0-P1/2
    [-1, 1, Conv, [128, 3, 2]], # 1-P2/4
    [-1, 3, C3, [128]],
    [-1, 1, Conv, [256, 3, 2]], # 3-P3/8
    [-1, 6, C3, [256]],
    [-1, 1, Conv, [512, 3, 2]], # 5-P4/16
    [-1, 9, C3, [512]],
    [-1, 1, Conv, [1024, 3, 2]], # 7-P5/32
    [-1, 3, C3, [1024]],
    [-1, 1, SPPF, [1024, 5]], # 9
  ]

# YOLOv5 v6.0 head
head: [
    [-1, 1, Conv, [512, 1, 1]],
    [-1, 1, nn.Upsample, [None, 2, "nearest"]],
    [[-1, 6], 1, Concat, [1]], # cat backbone P4
    [-1, 3, C3, [512, False]], # 13

    [-1, 1, Conv, [256, 1, 1]],
    [-1, 1, nn.Upsample, [None, 2, "nearest"]],
    [[-1, 4], 1, Concat, [1]], # cat backbone P3
    [-1, 3, C3, [256, False]], # 17 (P3/8-small)

    [-1, 1, Conv, [256, 3, 2]],
    [[-1, 14], 1, Concat, [1]], # cat head P4
    [-1, 3, C3, [512, False]], # 20 (P4/16-medium)

    [-1, 1, Conv, [512, 3, 2]],
    [[-1, 10], 1, Concat, [1]], # cat head P5
    [-1, 3, C3, [1024, False]], # 23 (P5/32-large)

    [[17, 20, 23], 1, Detect, [nc, anchors]], # Detect(P3, P4, P5)
  ]

## 5.Train

In [38]:
%pwd

'/content/drive/MyDrive/yolo/pklot detection/yolov5/models'

In [39]:
%cd /content/drive/MyDrive/yolo/pklot detection/yolov5

/content/drive/MyDrive/yolo/pklot detection/yolov5


In [40]:
%pwd

'/content/drive/MyDrive/yolo/pklot detection/yolov5'

In [41]:
!python train.py --img 640 --batch 32 --epochs 100 --data ./pklot/data.yaml --cfg ./models/custom_yolov5s.yaml --weights '' --name pklot_result --cache

       4/99       8.9G    0.05345     0.2338   0.005919       1197        640: 100% 272/272 [08:23<00:00,  1.85s/it]
                 Class     Images  Instances          P          R      mAP50   mAP50-95: 100% 39/39 [00:40<00:00,  1.03s/it]
                   all       2483     143316      0.953      0.958      0.972      0.564

      Epoch    GPU_mem   box_loss   obj_loss   cls_loss  Instances       Size
       5/99       8.9G    0.05073     0.2229   0.005544       1548        640: 100% 272/272 [08:14<00:00,  1.82s/it]
                 Class     Images  Instances          P          R      mAP50   mAP50-95: 100% 39/39 [00:42<00:00,  1.10s/it]
                   all       2483     143316      0.958      0.968      0.978      0.693

      Epoch    GPU_mem   box_loss   obj_loss   cls_loss  Instances       Size
       6/99       8.9G    0.04831     0.2142   0.005297       1096        640: 100% 272/272 [08:20<00:00,  1.84s/it]
                 Class     Images  Instances          P      

In [ ]:
Image(filename="",width = 600)

In [ ]:
!python val.py --weights /content/drice/MyDrive/yolo/yolov5/runs/train/pothole_result/weights/best.pt --data /content/drice/MyDrive/yolo/yolov5/pothole/data.yaml --img 640 --iou 0.65 --half